In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('script-bag-of-words.json')

In [3]:
df.tail()

,episodeAlt,seasonNum,episodeNum,episodeTitle,text
68,S8E2,8,2,A Knight of the Seven Kingdoms,"[{'name': 'Daenerys Targaryen', 'text': 'About..."
69,S8E3,8,3,The Long Night,"[{'name': 'Northman #1', 'text': 'Oi!'}, {'nam..."
70,S8E4,8,4,The Last of the Starks,"[{'name': 'Jon Snow', 'text': 'And Everyone It..."
71,S8E5,8,5,The Bells,"[{'name': 'Lord Varys', 'text': 'And? Come Not..."
72,S8E6,8,6,The Iron Throne,"[{'name': 'Tyrion Lannister', 'text': 'I'll fi..."


In [4]:
df.iloc[-1]['text']

[{'name': 'Tyrion Lannister', 'text': "I'll find later. you"},
 {'name': 'Jon Snow', 'text': "It's Let me men not safe. send some with you."},
 {'name': 'Tyrion Lannister', 'text': "I'm alone. going"},
 {'name': 'Grey Worm',
  'text': 'Daenerys I In Queen, Targaryen, die. name of one sentence the the to true you'},
 {'name': 'Jon Snow',
  'text': "Grey It's These Worm! are men over. prisoners."},
 {'name': 'Grey Worm',
  'text': "It Queen's are defeated. enemies is not over the until"},
 {'name': 'Davos Seaworth',
  'text': "How They're be? defeated do knees. more much on their them to want you"},
 {'name': 'Grey Worm', 'text': 'They are breathing.'},
 {'name': 'Davos Seaworth', 'text': 'Look We around friend. won. you,'},
 {'name': 'Grey Worm', 'text': "I commands, my not obey queen's yours."},
 {'name': 'Jon Snow', 'text': "And Queen's are commands? the what"},
 {'name': 'Grey Worm',
  'text': '"Kill Cersei Lannister." These They all are chose fight follow for free her. men. to who'}

In [9]:
dialouge = {}
for index,row in df.iterrows():
    for item in row['text']:
        if item['name'] in dialouge:
             # append
            dialouge[item['name']] = dialouge[item['name']] + item['text']
        else:
                dialouge[item['name']] = item['text']+ " "
           

In [10]:
len(dialouge)

817

In [11]:
new_df = pd.DataFrame()
new_df['character'] = dialouge.keys()
new_df['words'] = dialouge.values()

In [12]:
new_df.iloc[:,0:3].head()

,character,words
0,Will,"Easy, boy. I've I've Wildlings a a do ever in ..."
1,Waymar Royce,One They're What a and another before d'you ea...
2,Gared,Wall. We back head should the to Our They We W...
3,Jon Snow,Father's Go on. watching. And mother. yourBran...
4,Septa Mordane,"Fine Well always. as done. work, I Quite beaut..."


In [13]:
new_df['num_words'] = new_df['words'].apply(lambda x:len(x.split()))

In [14]:
new_df = new_df.sort_values('num_words',ascending=False)

In [15]:
new_df = new_df.head(100)

In [16]:
new_df.shape

(100, 3)

In [17]:
new_df.head()

,character,words,num_words
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words = 'english')


In [19]:
embeddings = cv.fit_transform(new_df['words']).toarray()

In [20]:
from sklearn.manifold import TSNE

In [21]:
tsne = TSNE(n_components=2,verbose=1,random_state=123)
z = tsne.fit_transform(embeddings)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 100 samples in 0.000s...
[t-SNE] Computed neighbors for 100 samples in 0.120s...
[t-SNE] Computed conditional probabilities for sample 100 / 100
[t-SNE] Mean sigma: 14.377887
[t-SNE] KL divergence after 250 iterations with early exaggeration: 62.945984
[t-SNE] KL divergence after 1000 iterations: 0.290837


In [23]:
new_df['x'] = z.T[0]
new_df['y'] = z.T[1]
#new_df['z'] = z.T[2]


In [24]:
new_df

,character,words,num_words,x,y
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924,-4.242916,-1.267377
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294,-4.014627,-1.144803
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488,-4.038165,-0.879828
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202,-3.363811,-1.224563
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823,-3.902705,-0.967790
...,...,...,...,...,...
132,Rickard Karstark,Castle I'll King North! Red The They and can c...,466,-0.395516,4.384218
57,Syrio Forel,"Tomorrow You are at be boy. here late, midday....",462,0.924658,3.751825
112,Kevan Lannister,Tyrion. Catelyn Golden Jaime Lords River River...,460,0.499867,3.802191
365,Mace Tyrell,"From Grace, House Margaery May Reach, Tyrell Y...",450,0.581692,3.655400


In [ ]:
import plotly.express as px
flg = px.scatter(new_df.head(25),x="x",y="y", color="character")
flg.show()